# L3: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 'crewai[tools]'
```

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, API and LLM

In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os
import dotenv
from rich.pretty import pprint as print

dotenv.load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Role Playing, Focus and Cooperation

In [5]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [6]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [7]:
from crewai_tools import ScrapeWebsiteTool, \
                         WebsiteSearchTool

/Users/leesean/agentic_workflows/venv/lib/python3.12/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [8]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [9]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [10]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [14]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=False,
  memory=True
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [15]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Task: DeepLearningAI just reached out with a super important ask:                                              │
│  I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you   │
│  provide guidance?                                                                                              │
│                                                                                                                 │
│  Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide     │
│  the best support possible.You must strive to provide a complete and accurate response to the customer's        │
│  inquiry.                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Thought: Thought: I need to provide comprehensive guidance on adding memory to a Crew using CrewAI for         │
│  DeepLearningAI, ensuring a detailed and informative response.                                                  │
│                                                                                                                 │
│  Using Tool: [Read website content]                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{}"                                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Introduction - CrewAI CrewAI home page English Search... ⌘ K Ask AI Start Cloud Trial crewAIInc / crewAI       │
│  crewAIInc / crewAI Search... Navigation Get Started Introduction Documentation Enterprise API Reference        │
│  Examples Website Forum Crew GPT Releases Get Started Introduction Installation Quickstart Guides Strategy      │
│  Agents Crews Flows Advanced Core Concepts Agents Tasks Crews Flows Knowledge LLMs Processes Collaboration      │
│  Training Memory Reasoning Planning Testing CLI Tools Event Listeners MCP Integration MCP Servers as Tools in   │
│  CrewAI Stdio Transport SSE Transport Streamable HTTP Transport Connecting to Multiple MCP Servers MCP          │
│  Security Considerations Tools Tools Overview File & Document Web Scraping & Browsing Search & Research         │
│  Database & Data AI & Machine Learning Cloud & Storage Automation & Integration Observability Overview          │
│  AgentOps Integration Arize Phoenix Langfuse Integration Langtrace Integration Maxim Integration MLflow         │
│  Integration Neatlogs Integration OpenLIT Integration Opik Integration Patronus AI Evaluation Portkey           │
│  Integration Weave Integration Learn Overview Strategic LLM Selection Guide Conditional Tasks Coding Agents     │
│  Create Custom Tools Custom LLM Implementation Custom Manager Agent Customize Agents Image Generation with      │
│  DALL-E Force Tool Output as Result Hierarchical Process Human Input on Execution Kickoff Crew Asynchronously   │
│  Kickoff Crew for Each Connect to any LLM Using Multimodal Agents Replay Tasks from Latest Crew Kickoff         │
│  Sequential Processes Using Annotations in crew.py Telemetry Telemetry On this page What is CrewAI? How Crews   │
│  Work How It All Works Together Key Features How Flows Work Key Capabilities When to Use Crews vs. Flows        │
│  Decision Framework Why Choose CrewAI? Ready to Start Building? Get Started Introduction Copy page Build AI     │
│  agent teams that work together to tackle complex tasks ​ What is CrewAI?                                        │
│  CrewAI is a lean, lightning-fast Python framework built entirely from scratch—completely independent of        │
│  LangChain or other agent framework...                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  To add memory to your Crew using CrewAI, you can leverage the capabilities of the framework to create          │
│  specialized AI agents with defined roles and expertise. The CrewAI framework allows you to equip these agents  │
│  with custom tools and APIs to interact with external services and data sources. The intelligent collaboration  │
│  feature of CrewAI enables agents to work together, sharing insights and coordinating tasks to achieve complex  │
│  objectives. Additionally, you can define sequential or parallel workflows within the Crew to manage task       │
│  dependencies efficiently.                                                                                      │
│                                                                                                                 │
│  When it comes to memory specifically, you can design your AI agents to store and retrieve information during   │
│  the execution of tasks by utilizing memory management techniques within the agents. This can include storing   │
│  relevant data points, past interactions, or learned patterns to enhance decision-making and task performance.  │
│  By incorporating memory into your AI agents within the Crew, you can create more adaptive and knowledgeable    │
│  responses to varying scenarios.                                                                                │
│                                                                                                                 │
│  Overall, by structuring your Crew with memory-enhanced AI agents, you can improve the efficiency, accuracy,    │
│  and adaptability of your AI team in tackling complex tasks. For more detailed guidance on implementing memory  │
│  within your Crew using CrewAI, you can refer to the documentation and resources provided on the CrewAI         │
│  website.                                                                                                       │
│                                                                                                                 │
│  Reference: https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/                                     │
│  Entities:                                                                                                      │
│  - CrewAI(Framework): An artificial intelligence framework designed for creating specialized AI agents with     │
│  defined roles and expertise.                                                                                   │
│  - CrewAI(Tool): A framework for creating specialized AI agents with defined roles and expertise.               │
│  - Memory(Functionality): The ability of AI agents within CrewAI to store and retrieve information, enhancing   │
│  decision-making and task performance.                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/leesean/agentic_workflows/venv/lib/python3.12/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
/Users/leesean/agentic_workflows/venv/lib/python3.12/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│  Task: Review the response drafted by the Senior Support Representative for DeepLearningAI's inquiry. Ensure    │
│  that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer    │
│  support.                                                                                                       │
│  Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly    │
│  tone.                                                                                                          │
│  Check for references and sources used to  find the information, ensuring the response is well-supported and    │
│  leaves no questions unanswered.                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Hi there!                                                                                                      │
│                                                                                                                 │
│  Thank you for reaching out to DeepLearningAI with your inquiry about adding memory to your Crew using CrewAI.  │
│  I'm excited to provide you with a comprehensive explanation on how you can incorporate memory functionality    │
│  into your AI agents to enhance their capabilities within the framework.                                        │
│                                                                                                                 │
│  To increase the memory capacity of your AI agents within CrewAI, you can leverage the framework's features to  │
│  create specialized agents with defined roles and expertise. These agents can be equipped with custom tools     │
│  and APIs to interact with external services and data sources effectively. By enabling intelligent              │
│  collaboration among the agents, they can share insights and coordinate tasks to achieve complex objectives     │
│  seamlessly. Additionally, you have the flexibility to define sequential or parallel workflows within your      │
│  Crew to manage task dependencies efficiently.                                                                  │
│                                                                                                                 │
│  Specifically focusing on memory, you can design your AI agents to store and retrieve information crucial for   │
│  task execution. By implementing memory management techniques within the agents, you can save relevant data     │
│  points, past interactions, or learned patterns. This approach significantly enhances the decision-making       │
│  process and improves task performance by enabling the agents to deliver more adaptive and knowledgeable        │
│  responses across various scenarios.                                                                            │
│                                                                                                                 │
│  By structuring your Crew with memory-enhanced AI agents, you can enhance the efficiency, accuracy, and         │
│  adaptability of your AI team in addressing challenging tasks. For further guidance on integrating memory into  │
│  your Crew leveraging CrewAI, you may explore the documentation and resources available on the CrewAI website.  │
│                                                                                                                 │
│  If you are looking for more detailed steps or practical examples on how to implement memory within CrewAI,     │
│  you can refer to the specific guide provided in the following link: [CrewAI Documentation on Adding Memory     │
│  Functionality](https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/). This resource will provide    │
│  you with a deeper understanding and hands-on instructions for enhancing the memory capacity of your AI agents  │
│  within the framework.                                                                                          │
│                                                                                                                 │
│  In conclusion, by enhancing your AI agents within Crew

/Users/leesean/agentic_workflows/venv/lib/python3.12/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
/Users/leesean/agentic_workflows/venv/lib/python3.12/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
